<a href="https://colab.research.google.com/github/felipefreitas93/mestrado_ppgee/blob/master/xlnet_large_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import datetime
import json
import pprint
import random
import string
import sys
import tensorflow as tf

#install dependencies
import os
import csv
import tensorflow as tf
import pandas as pd  
import subprocess
import sys

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.
  
try:
  tf.gfile.DeleteRecursively('gs://deep_learning_bucket_mestrado/xlnet/')
except:
  pass


git_url = "https://github.com/felipefreitas93/Colab-XLNet-FineTuning.git"
os.system("git clone "+git_url)
%cd Colab-XLNet-FineTuning


!git pull origin master
#Use if you have updated git repo and want changes to reflect

!git clone https://github.com/felipefreitas93/NLPdatasets.git
  
  
print(os.environ)

#0.029474180617778827, 0.058948361235557654, 0.11789672247111531, 0.23579344494223062, 0.47158688988446124
dataset_used='MPQA'
FRAC = 0.029474180617778827

TASK_DATA_DIR = f'NLPdatasets/{dataset_used}'
train = pd.read_csv(TASK_DATA_DIR + '/train.tsv', sep='\t', names=['y','x']).dropna().sample(frac=FRAC)
dataset_len = train.shape[0]/FRAC
train.to_csv(TASK_DATA_DIR+ '/train.tsv', sep='\t', index=False, header=False)

NUM_TRAIN_STEPS = 5.12*dataset_len*FRAC/32 #batch size 5.12
WARMUP_STEPS = 0.125*5.12*dataset_len*FRAC/32 #batch size 5.12

TPU address is grpc://10.0.49.146:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 5948409670868560008),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 12876730059906198617),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 14833259727230941022),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 12788793690885660110),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 14991471429767226403),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0

In [0]:
!pip install sentencepiece

     |████████████████████████████████| 1.0MB 3.5MB/s 


In [0]:
TASK = 'IMDB' #@param {type:"string"}

print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))
!ls $TASK_DATA_DIR

BUCKET = 'deep_learning_bucket_mestrado' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/xlnet/output/{}'.format(BUCKET, TASK)
MODEL_DIR = 'gs://{}/xlnet/model/{}'.format(BUCKET, TASK)

tf.gfile.MakeDirs(OUTPUT_DIR)
tf.gfile.MakeDirs(MODEL_DIR)

print('***** Model output directory: {} *****'.format(OUTPUT_DIR))



***** Task data directory: NLPdatasets/MPQA *****
dev.tsv  train.tsv
***** Model output directory: gs://deep_learning_bucket_mestrado/xlnet/output/IMDB *****


In [0]:
os.system("wget https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip")
os.system("unzip cased_L-24_H-1024_A-16.zip")
!ls

cased_L-24_H-1024_A-16.zip  modeling.py        run_squad.py
classifier_utils.py	    model_utils.py     scripts
data_utils.py		    NLPdatasets        squad_utils.py
function_builder.py	    notebooks	       tpu_estimator.py
gpu_utils.py		    prepro_utils.py    train_gpu.py
__init__.py		    README.md	       train.py
LICENSE			    run_classifier.py  xlnet_cased_L-24_H-1024_A-16
misc			    run_race.py        xlnet.py


In [0]:
%cd xlnet_cased_L-24_H-1024_A-16
!ls

/content/Colab-XLNet-FineTuning/xlnet_cased_L-24_H-1024_A-16
spiece.model	   xlnet_model.ckpt.data-00000-of-00001  xlnet_model.ckpt.meta
xlnet_config.json  xlnet_model.ckpt.index


In [0]:
file_names = os.listdir(os.getcwd())
print(file_names)

['xlnet_model.ckpt.index', 'xlnet_config.json', 'xlnet_model.ckpt.meta', 'xlnet_model.ckpt.data-00000-of-00001', 'spiece.model']


In [0]:
for file_name in file_names:
  print(file_name)
  os.system("gsutil cp "+ file_name + " " + MODEL_DIR)
os.system("gsutil ls " + MODEL_DIR)
%cd ..

xlnet_model.ckpt.index
xlnet_config.json
xlnet_model.ckpt.meta
xlnet_model.ckpt.data-00000-of-00001
spiece.model
/content/Colab-XLNet-FineTuning


In [0]:
os.system("gsutil cp -r " + MODEL_DIR + "/spiece.model spiece.model")
!ls

cased_L-24_H-1024_A-16.zip  model_utils.py     spiece.model
classifier_utils.py	    NLPdatasets        squad_utils.py
data_utils.py		    notebooks	       tpu_estimator.py
function_builder.py	    prepro_utils.py    train_gpu.py
gpu_utils.py		    README.md	       train.py
__init__.py		    run_classifier.py  xlnet_cased_L-24_H-1024_A-16
LICENSE			    run_race.py        xlnet.py
misc			    run_squad.py
modeling.py		    scripts


In [0]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
MAX_SEQ_LENGTH = 256


# Model configs
SAVE_CHECKPOINTS_STEPS = 4500
NUM_ITERATIONS = 500

In [0]:
train_command = "python run_classifier.py \
  --use_tpu=True \
  --use_colab_tpu=True \
  --do_train=True \
  --do_eval=False \
  --task_name="+TASK.lower()+" \
  --data_dir=./"+TASK_DATA_DIR+" \
  --output_dir="+OUTPUT_DIR+" \
  --model_dir="+MODEL_DIR+" \
  --uncased=False \
  --tpu_address="+TPU_ADDRESS+"  \
  --spiece_model_file=./spiece.model \
  --model_config_path="+MODEL_DIR+"/xlnet_config.json \
  --init_checkpoint="+MODEL_DIR+"/xlnet_model.ckpt \
  --max_seq_length="+str(MAX_SEQ_LENGTH)+" \
  --train_batch_size="+str(TRAIN_BATCH_SIZE)+" \
  --eval_batch_size="+str(EVAL_BATCH_SIZE)+" \
  --num_hosts=1 \
  --num_core_per_host=8 \
  --learning_rate="+str(LEARNING_RATE)+" \
  --train_steps="+str(int(NUM_TRAIN_STEPS))+" \
  --warmup_steps="+str(int(WARMUP_STEPS))+" \
  --save_steps="+str(SAVE_CHECKPOINTS_STEPS)+" \
  --iterations="+ str(NUM_ITERATIONS)

print(train_command)


python run_classifier.py   --use_tpu=True   --use_colab_tpu=True   --do_train=True   --do_eval=False   --task_name=imdb   --data_dir=./NLPdatasets/MPQA   --output_dir=gs://deep_learning_bucket_mestrado/xlnet/output/IMDB   --model_dir=gs://deep_learning_bucket_mestrado/xlnet/model/IMDB   --uncased=False   --tpu_address=grpc://10.0.49.146:8470    --spiece_model_file=./spiece.model   --model_config_path=gs://deep_learning_bucket_mestrado/xlnet/model/IMDB/xlnet_config.json   --init_checkpoint=gs://deep_learning_bucket_mestrado/xlnet/model/IMDB/xlnet_model.ckpt   --max_seq_length=256   --train_batch_size=32   --eval_batch_size=8   --num_hosts=1   --num_core_per_host=8   --learning_rate=2e-05   --train_steps=40   --warmup_steps=5   --save_steps=4500   --iterations=500


In [0]:
%%time
!{train_command}




W0914 18:12:15.483564 140232853186432 deprecation_wrapper.py:119] From run_classifier.py:640: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0914 18:12:15.483804 140232853186432 deprecation_wrapper.py:119] From run_classifier.py:640: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0914 18:12:15.483958 140232853186432 deprecation_wrapper.py:119] From run_classifier.py:664: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

I0914 18:12:17.348760 140232853186432 utils.py:141] NumExpr defaulting to 2 threads.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an 

##Run Evaluation

In [0]:
eval_command = "python run_classifier.py \
  --use_tpu=True \
  --use_colab_tpu=True \
  --do_train=False \
  --do_eval=True \
  --eval_all_ckpt=True \
  --task_name="+TASK.lower()+" \
  --data_dir=./"+TASK_DATA_DIR+" \
  --output_dir="+OUTPUT_DIR+" \
  --model_dir="+OUTPUT_DIR+" \
  --uncased=False \
  --tpu_address="+TPU_ADDRESS+"  \
  --spiece_model_file=./spiece.model \
  --model_config_path="+MODEL_DIR+"/xlnet_config.json \
  --init_checkpoint="+MODEL_DIR+"/xlnet_model.ckpt \
  --max_seq_length="+str(MAX_SEQ_LENGTH)+" \
  --train_batch_size="+str(TRAIN_BATCH_SIZE)+" \
  --eval_batch_size="+str(EVAL_BATCH_SIZE)+" \
  --num_hosts=1 \
  --num_core_per_host=8 \
  --learning_rate=2e-5 \
  --train_steps="+str(int(NUM_TRAIN_STEPS))+" \
  --warmup_steps="+str(int(WARMUP_STEPS))+" \
  --save_steps="+str(SAVE_CHECKPOINTS_STEPS)+" \
  --iterations="+ str(NUM_ITERATIONS)

print(eval_command)


python run_classifier.py   --use_tpu=True   --use_colab_tpu=True   --do_train=False   --do_eval=True   --eval_all_ckpt=True   --task_name=imdb   --data_dir=./NLPdatasets/MPQA   --output_dir=gs://deep_learning_bucket_mestrado/xlnet/output/IMDB   --model_dir=gs://deep_learning_bucket_mestrado/xlnet/output/IMDB   --uncased=False   --tpu_address=grpc://10.0.49.146:8470    --spiece_model_file=./spiece.model   --model_config_path=gs://deep_learning_bucket_mestrado/xlnet/model/IMDB/xlnet_config.json   --init_checkpoint=gs://deep_learning_bucket_mestrado/xlnet/model/IMDB/xlnet_model.ckpt   --max_seq_length=256   --train_batch_size=32   --eval_batch_size=8   --num_hosts=1   --num_core_per_host=8   --learning_rate=2e-5   --train_steps=40   --warmup_steps=5   --save_steps=4500   --iterations=500


In [0]:
%%time
!{eval_command}




W0914 18:17:58.183635 140634917963648 deprecation_wrapper.py:119] From run_classifier.py:640: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0914 18:17:58.183893 140634917963648 deprecation_wrapper.py:119] From run_classifier.py:640: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0914 18:17:58.184048 140634917963648 deprecation_wrapper.py:119] From run_classifier.py:664: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

I0914 18:18:00.076092 140634917963648 utils.py:141] NumExpr defaulting to 2 threads.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an 